# Conversational Memory

In [ ]:
!pip install -qU langchain langchain-openai langchain-google-genai

In [1]:
import os
from getpass import getpass

In [2]:
os.environ['OPENAI_API_KEY'] = getpass()

In [3]:
os.environ['GOOGLE_API_KEY'] = getpass()

## Without Memory

In [4]:
from langchain_openai import ChatOpenAI
gpt3_model = ChatOpenAI(model = "gpt-3.5-turbo-0125" , temperature=0.7)


from langchain_google_genai import ChatGoogleGenerativeAI
gemini_model = ChatGoogleGenerativeAI(model = "gemini-pro")

In [ ]:
gpt3_model.invoke('Who is the first black president of USA?')

In [ ]:
gpt3_model.invoke("When was he born?")

In [ ]:
gemini_model.invoke("Who is the first African-American president of USA?")

In [ ]:
gemini_model.invoke("When was he born?")

In [ ]:
gemini_model.invoke("When did World War I start?")

## Adding Memory

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory

In [ ]:
# memory = ConversationBufferMemory(k = 100)
memory = ConversationSummaryMemory()

conversation = ConversationChain(
    llm=gpt3_model,
    memory = memory
)

In [ ]:
conversation.predict(input='Who is the first black president of USA?')

In [ ]:
conversation.predict(input = "When was he born?")

In [ ]:
conversation.predict(input = "When did his tenure end?")

# RAG

In [ ]:
!pip install pypdf

In [ ]:
from langchain.document_loaders import PyPDFLoader, TextLoader, NotionDirectoryLoader

pdf_path = "./Rahul_Nihalani_Resume.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()
print(pages[0].page_content)

In [ ]:
!pip install -qU tiktoken faiss-cpu

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS

faiss_index = FAISS.from_documents(pages, embeddings)

In [ ]:
import re

query = "What is the document about?"
num_results = 2  # Number of results you want to retrieve
search_type = "similarity"  # Choose between "similarity" and "hybrid"

results = faiss_index.search(query, search_type, k=num_results)

for result in results:
    output_text = re.sub(r'(?<=\w) (?=\w)', '', result.page_content)
    print(output_text)